## YIN
A step-by-step guide through the YIN [1] algorithm. This notebook is part of **libf0 -  A Python Library for F0-Estimation in Music Recordings**.

[1] Alain de Cheveigné and Hideki Kawahara. YIN, a fundamental frequency estimator for speech and music. Journal of the Acoustical Society of America (JASA), 111(4):1917–1930, 2002.

Available implementations:
* https://librosa.org/doc/main/generated/librosa.yin.html
* http://audition.ens.fr/adc/
* https://github.com/patriceguyot/Yin
* https://zenodo.org/record/1220947
* https://aubio.org/doc/0.4.7/pitch_8h.html

In [ ]:
import numpy as np
import librosa

import IPython.display as ipd
import matplotlib.pyplot as plt

In [ ]:
# load demo audio (a throat microphone recording of a soprano singer)
fn_wav = "../data/DCS_LI_QuartetB_Take03_S1_LRX_excerpt.wav"
x, Fs = librosa.load(fn_wav, sr=22050)
ipd.display(ipd.Audio(x, rate=Fs, normalize=True)) # audio playback

# set all parameters for YIN
N = 2048 # frame size of the algorithm in samples
H = 256 # hop size of the algorithm in samples
F_min = 55.0 # minimum frequency of interest in Hz
F_max = 1760.0 # maximum frequency of interest in Hz
threshold = 0.15 # threshold for cumulative mean normalized difference function

# extract an example frame for visualization
example_frame = x[1*Fs:1*Fs+512]

In the following, we visualize the YIN algorithm for an example frame from our input signal. First, we calculate the cumulative mean normalized difference function (CMNDF).


**TODO:**
* What is the "cumulative mean" of a difference function?
    1. plot audio in time domain
    2. difference function of this signal
    3. cmndf without thresholding (try: "cmndf spectrogram" over multiple frames by using color for function value)
* Where is the global minimum? Could be the octave, etc.

In [ ]:
def cumulative_mean_normalized_difference_function(frame, lag_max):
    """
    Computes Cumulative Mean Normalized Difference Function (CMNDF).

    Parameters
    ----------
    frame : ndarray
        Audio frame
    lag_max : int
        Maximum expected lag in the CMNDF

    Returns
    -------
    cmndf : ndarray
        Cumulative Mean Normalized Difference Function
    """

    cmndf = np.zeros(lag_max+1)  # Initialize CMNDF
    cmndf[0] = 1
    diff_mean = 0

    for tau in range(1, lag_max+1):
        # Difference function
        diff = np.sum((frame[0:-tau] - frame[0 + tau:]) ** 2)
        # Iterative mean of the difference function
        diff_mean = diff_mean*(tau-1)/tau + diff/tau

        cmndf[tau] = diff / (diff_mean + np.finfo(np.float64).eps)

    return cmndf

lag_min = max(int(np.ceil(Fs / F_max)), 1)  # lag of maximal frequency in samples
lag_max = int(np.ceil(Fs / F_min))  # lag of minimal frequency in samples

cmndf = cumulative_mean_normalized_difference_function(example_frame, lag_max)

plt.figure(figsize=(6.3, 2))
plt.plot(cmndf, color='dimgrey')
plt.axhline(threshold, 0, 1, color="red", linestyle=':')
plt.grid()
plt.xlim((0, 400))
plt.xlabel('Lag (samples)')
plt.ylabel('CMNDF')
plt.tight_layout()

Now we can search for the smallest value below the threshold as the lag of our F0 estimate. The estimate is refined beyond the sampling resolution by using parabolic interpolation of the CMNDF.

**TODO:**
* Introduce threshold to mitigate the global minimum problem
* Visualize parabolic interpolation

In [ ]:
def parabolic_interpolation(y1, y2, y3):
    """
    Parabolic interpolation of an extremal value given three samples with equal spacing on the x-axis.
    The middle value y2 is assumed to be the extremal sample of the three.

    Parameters
    ----------
    y1: f(x1)
    y2: f(x2)
    y3: f(x3)

    Returns
    -------
    x_interp: Interpolated x-value (relative to x3-x2)
    y_interp: Interpolated y-value, f(x_interp)
    """

    a = np.finfo(np.float64).eps + (y1 + y3 - 2 * y2) / 2
    b = (y3 - y1) / 2
    x_interp = -b / (2 * a)
    y_interp = y2 - (b ** 2) / (4 * a)

    return x_interp, y_interp

def absolute_thresholding(cmndf, threshold, lag_min, lag_max, parabolic_interp=True):
    """
    Absolute thresholding:
    Set an absolute threshold and choose the smallest value of tau that gives a minimum of d' deeper than that
    threshold. If none is found, the global minimum is chosen instead.

    Parameters
    ----------
    cmndf : ndarray
        Cumulative Mean Normalized Difference Function
    threshold : float
        Threshold
    lag_min : float
        Minimal lag
    lag_max : float
        Maximal lag
    parabolic_interp : bool
        Switch to activate/deactivate parabolic interpolation

    Returns
    -------

    """

    # take shortcut if search range only allows for one possible lag
    if lag_min == lag_max:
        return lag_min

    # find local minima below absolute threshold in interval [lag_min:lag_max]
    local_min_idxs = (np.argwhere((cmndf[1:-1] < cmndf[0:-2]) & (cmndf[1:-1] < cmndf[2:]))).flatten() + 1
    below_thr_idxs = np.argwhere(cmndf[lag_min:lag_max] < threshold).flatten() + lag_min
    # numba compatible intersection of indices sets
    min_idxs = np.unique(np.array([i for i in local_min_idxs for j in below_thr_idxs if i == j]))

    # if no local minima below threshold are found, return global minimum
    if not min_idxs.size:
        return np.argmin(cmndf[lag_min:lag_max]) + lag_min

    # find first local minimum
    lag = np.min(min_idxs)  # choose first local minimum

    # Optional: Parabolic Interpolation of local minima
    if parabolic_interp:
        lag_corr, cmndf[lag] = parabolic_interpolation(cmndf[lag-1], cmndf[lag], cmndf[lag+1])
        lag += lag_corr

    return lag

lag_est = absolute_thresholding(cmndf, threshold, lag_min, lag_max, parabolic_interp=True)

print("Lag estimate (first interpolated minimum below the threshold): %.2f samples\nResulting F0 estimate: %.2f Hz" % (lag_est, Fs / lag_est))

We can refine this estimate by thresholding the CMNDF again but focussing on the vicinity of our previous estimate (+/- 25 cents). This is step 6 in [1].

**TODO:**
* Show that this refinement has an effect (e.g. zooming in a specific region)

In [ ]:
tol_cents = 25
lag_min_local = int(np.round(Fs / ((Fs / lag_est) * 2 ** (tol_cents/1200))))
if lag_min_local < lag_min:
    lag_min_local = lag_min
lag_max_local = int(np.round(Fs / ((Fs / lag_est) * 2 ** (-tol_cents/1200))))
if lag_max_local > lag_max:
    lag_max_local = lag_max
lag_new = absolute_thresholding(cmndf, threshold=np.inf, lag_min=lag_min_local, lag_max=lag_max_local, parabolic_interp=True)

print("Refined lag estimate (first interpolated minimum below the threshold): %.2f s\nResulting F0 estimate: %.2f Hz" % (lag_est, Fs / lag_est))

Finally we can calculate the aperiodicity of the signal as an estimate of the confidence in the F0 estimate.

**TODO:**
* visualize ap. by plotting repeated frame + time domain signal in the vicinity
* then show their correlation (for a positive and negative example)

In [ ]:
def aperiodicity(frame, lag_est):
    """
    Compute aperiodicity of given frame (serves as indicator for reliability or voicing detection).

    Parameters
    ----------
    frame : ndarray
        Frame
    lag_est : float
        Estimated lag

    Returns
    -------
    ap: float
        Aperiodicity (the lower, the more reliable the estimate)
    """

    lag_int = int(np.floor(lag_est))  # uncorrected period estimate
    frac = lag_est - lag_int  # residual

    # Pad frame to insure constant size
    frame_pad = np.concatenate((frame, np.flip(frame)))  # mirror padding

    # Shift frame by estimated period
    if frac == 0:
        frame_shift = frame_pad[lag_int:lag_int+len(frame)]
    else:
        # linear interpolation between adjacent shifts
        frame_shift = (1 - frac) * frame_pad[lag_int:lag_int+len(frame)] + \
                      frac * frame_pad[lag_int+1:lag_int+1+len(frame)]

    pwr = (np.mean(frame ** 2) + np.mean(frame_shift ** 2)) / 2  # average power over fixed and shifted frame
    res = np.mean((frame - frame_shift) ** 2) / 2  # residual power
    ap = res / (pwr + np.finfo(np.float64).eps)

    return ap

ap = aperiodicity(example_frame, lag_new)

# TODO: print nicely
print("Aperiodicity for this frame: %.4f" % ap)

The complete above process can now be repeated for each frame of the signal.

In [ ]:
x_pad = np.concatenate((np.zeros(N//2), x, np.zeros(N//2)))  # Add zeros for centered estimates
M = int(np.floor((len(x_pad) - N) / H)) + 1  # Compute number of estimates that will be generated
f0 = np.zeros(M)  # Estimated fundamental frequencies (0 for unspecified frames)
t = np.arange(M) * H / Fs  # Time axis
ap = np.zeros(M)  # Aperiodicity

assert lag_max <= N, "N needs to be <= Fs/F_min!"

for m in range(M):
    print(f"YIN Progress: {np.ceil(100*m/M).astype(int)}%", end='\r')
    # Take a frame from input signal
    
    frame = x_pad[m*H:m*H + N]

    # Cumulative Mean Normalized Difference Function
    cmndf = cumulative_mean_normalized_difference_function(frame, lag_max)

    # Absolute Thresholding
    lag_est = absolute_thresholding(cmndf, threshold, lag_min, lag_max, parabolic_interp=True)

    # Refine estimate by constraining search to vicinity of best local estimate (default: +/- 25 cents)
    tol_cents = 25
    lag_min_local = int(np.round(Fs / ((Fs / lag_est) * 2 ** (tol_cents/1200))))
    if lag_min_local < lag_min:
        lag_min_local = lag_min
    lag_max_local = int(np.round(Fs / ((Fs / lag_est) * 2 ** (-tol_cents/1200))))
    if lag_max_local > lag_max:
        lag_max_local = lag_max
    lag_new = absolute_thresholding(cmndf, threshold=np.inf, lag_min=lag_min_local, lag_max=lag_max_local,
                                    parabolic_interp=True)

    # Compute Fundamental Frequency Estimate
    f0[m] = Fs / lag_new

    # Compute Aperiodicity
    ap[m] = aperiodicity(frame, lag_new)

# Visualize F0-Trajectory and Aperiodicity
plt.figure(figsize=(9, 6))
plt.plot(t, f0, linestyle='', color='k', marker='.')
plt.ylim((0, 600))
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')

plt.figure(figsize=(9, 6))
plt.plot(t, ap, color='k')
plt.xlabel('Time (seconds)')
plt.ylabel('Aperiodicity')